In [64]:
# Class to colorize, bold, or underline output
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
# 

import pandas as pd
import numpy as np
# define header for our data, the UCI dataset does not have a header
headers = ["age", "workclass", "fnlwgt", "education", "education-num",
           "marital-status", "occupation", "relationship", "race",
           "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country",
           "income"]
# load the data from github repo and convert the values with '?' to NaN
url = "https://raw.githubusercontent.com/vbloise3/WhizLabsML/master/CensusIncome/CensusIncomeDataset.csv"
df = pd.read_csv(url, error_bad_lines=False, header=None, names=headers, na_values="null")
# head of the df
df.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [65]:
# Separate the features
features_df = df.drop('income', axis=1)
# Separate the target
target_df = df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]], axis=1)
features_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States


In [66]:
# What data types are in the dataset
features_df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
dtype: object

In [67]:
# Create a dataframe of only the categorical features
categorical_featuresDf = features_df.select_dtypes(include=['object']).copy()
categorical_featuresDf

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba
...,...,...,...,...,...,...,...,...
32556,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States
32557,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States
32558,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States
32559,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States


In [68]:
# Find any null value entries in the categorical features
categorical_features_NaN = categorical_featuresDf[categorical_featuresDf.isnull().any(axis=1)]
categorical_features_NaN

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
14,Private,Assoc-voc,Married-civ-spouse,Craft-repair,Husband,Asian-Pac-Islander,Male,NaN
27,NaN,Some-college,Married-civ-spouse,NaN,Husband,Asian-Pac-Islander,Male,South
38,Private,Some-college,Married-civ-spouse,Sales,Husband,White,Male,NaN
51,Private,HS-grad,Never-married,Other-service,Own-child,White,Female,NaN
61,NaN,7th-8th,Married-spouse-absent,NaN,Not-in-family,White,Male,NaN
...,...,...,...,...,...,...,...,...
32530,NaN,Bachelors,Married-civ-spouse,NaN,Wife,White,Female,United-States
32531,NaN,Bachelors,Never-married,NaN,Not-in-family,Asian-Pac-Islander,Female,United-States
32539,NaN,Doctorate,Married-civ-spouse,NaN,Husband,White,Male,United-States
32541,NaN,HS-grad,Separated,NaN,Not-in-family,Black,Female,United-States


In [69]:
# Which features hava a NaN value?
categorical_featuresDf.columns[categorical_featuresDf.isna().any()].tolist()

['workclass', 'occupation', 'native-country']

In [70]:
#Impute the NaN values using scikit-learn SimpleImpute Class
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
# Impute workclass values
imputer = imputer.fit(features_df[['workclass']])
features_df['workclass'] = imputer.transform(features_df[['workclass']]).ravel()
# Impute occupation values
imputer = imputer.fit(features_df[['occupation']])
features_df['occupation'] = imputer.transform(features_df[['occupation']]).ravel()
# Impute native-country values
imputer = imputer.fit(features_df[['native-country']])
features_df['native-country'] = imputer.transform(features_df[['native-country']]).ravel()

# Recreate the dataframe of only the categorical features
categorical_featuresDf = features_df.select_dtypes(include=['object']).copy()

# Recheck to find any null value entries in the categorical features
categorical_features_NaN = categorical_featuresDf[categorical_featuresDf.isnull().any(axis=1)]
# Which features hava a NaN value?
categorical_featuresDf.columns[categorical_featuresDf.isna().any()].tolist()

[]

In [71]:
# Use binary encoding for the sex feature
from sklearn.preprocessing import LabelBinarizer

# How many differernt sex feature value types
print(color.BOLD + color.PURPLE + "\nHow many different sex feature types?" + color.END)
print(categorical_featuresDf["sex"].value_counts())

label_style = LabelBinarizer()
label_results = label_style.fit_transform(categorical_featuresDf["sex"])
print(color.BOLD + color.PURPLE + "\nLabelBinarizer of sex feature" + color.END)
data_sex = pd.DataFrame({'sex': label_results[:, 0]})
data_sex


How many different sex feature types?
 Male      21790
 Female    10771
Name: sex, dtype: int64

LabelBinarizer of sex feature


,sex
0,1
1,1
2,1
3,1
4,0
...,...
32556,0
32557,1
32558,0
32559,1


In [72]:
# Perform label encoding on workclass feature
from sklearn.preprocessing import LabelEncoder
label_work_class = LabelEncoder()
categorical_featuresDf["workclass_code"] = label_work_class.fit_transform(categorical_featuresDf["workclass"])
categorical_featuresDf[["workclass", "workclass_code"]].head(15)

,workclass,workclass_code
0,State-gov,6
1,Self-emp-not-inc,5
2,Private,3
3,Private,3
4,Private,3
5,Private,3
6,Private,3
7,Self-emp-not-inc,5
8,Private,3
9,Private,3


In [188]:
# Use one-hot encoding on the workclass feature

# How many differernt workclass feature value types
print(color.BOLD + color.PURPLE + "\nHow many different workclass feature types?" + color.END)
print(categorical_featuresDf["workclass"].value_counts())

# One-hot encode the workclass feature
pd.get_dummies(categorical_featuresDf, columns=["workclass"]).head()


How many different workclass feature types?
 Private             24532
 Self-emp-not-inc     2541
 Local-gov            2093
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64


,education,marital-status,occupation,relationship,race,sex,native-country,workclass_code,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay
0,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,6,0,0,0,0,0,0,1,0
1,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,5,0,0,0,0,0,1,0,0
2,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,3,0,0,0,1,0,0,0,0
3,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,3,0,0,0,1,0,0,0,0
4,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,3,0,0,0,1,0,0,0,0


In [187]:
# Use one-hot encoding on the marital-status feature

# How many differernt marital-status feature value types
print(color.BOLD + color.PURPLE + "\nHow many different marital-status feature types?" + color.END)
print(categorical_featuresDf["marital-status"].value_counts())

# One-hot encode the marital-status feature
pd.get_dummies(categorical_featuresDf, columns=["marital-status"]).head()


How many different marital-status feature types?
 Married-civ-spouse       14976
 Never-married            10683
 Divorced                  4443
 Separated                 1025
 Widowed                    993
 Married-spouse-absent      418
 Married-AF-spouse           23
Name: marital-status, dtype: int64


,workclass,education,occupation,relationship,race,sex,native-country,workclass_code,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed
0,State-gov,Bachelors,Adm-clerical,Not-in-family,White,Male,United-States,6,0,0,0,0,1,0,0
1,Self-emp-not-inc,Bachelors,Exec-managerial,Husband,White,Male,United-States,5,0,0,1,0,0,0,0
2,Private,HS-grad,Handlers-cleaners,Not-in-family,White,Male,United-States,3,1,0,0,0,0,0,0
3,Private,11th,Handlers-cleaners,Husband,Black,Male,United-States,3,0,0,1,0,0,0,0
4,Private,Bachelors,Prof-specialty,Wife,Black,Female,Cuba,3,0,0,1,0,0,0,0


In [171]:
# Use one-hot encoding on the native-country feature

# How many differernt native-country feature value types
print(color.BOLD + color.PURPLE + "\nHow many different native-country feature types?" + color.END)
print(categorical_featuresDf["native-country"].value_counts())

# One-hot encode the native-country feature
pd.get_dummies(categorical_featuresDf, columns=["native-country"]).head()


How many different native-country feature types?
 United-States                 29753
 Mexico                          643
 Philippines                     198
 Germany                         137
 Canada                          121
 Puerto-Rico                     114
 El-Salvador                     106
 India                           100
 Cuba                             95
 England                          90
 Jamaica                          81
 South                            80
 China                            75
 Italy                            73
 Dominican-Republic               70
 Vietnam                          67
 Guatemala                        64
 Japan                            62
 Poland                           60
 Columbia                         59
 Taiwan                           51
 Haiti                            44
 Iran                             43
 Portugal                         37
 Nicaragua                        34
 Peru                    

,workclass,education,marital-status,occupation,relationship,race,sex,workclass_code,native-country_ Cambodia,native-country_ Canada,native-country_ China,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Holand-Netherlands,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
